# Computing the bicharacter of diagonal harmonic polynomials


This package was developped at the occasion of a collaboration with François Bergeron to study the character of the space $H_{k,n}$ of diagonal harmonic polynomials on $k$ sets of $n$ variables, as a graded bi $GL_k$-$S_n$  module. The main achievement is a calculation of this character for $n=6$ and $r=5$ (and therefore any $r$) in 45 minutes (and ~15Gb), which led to conjectures on its general value.

This is a potentially huge calculation ($H_{5,6}$ is of dimension $5.8 \cdot10^5$), involving polynomials of degree up to $15$ in $30$ variables with thousands of terms. So we had to combine several strategies that each reduced the time and/or memory complexity by one or two orders of magnitude. As much as possible, the code was written to be modular and reusable.

In this notebook, we introduce the problem and describe the main computation strategies, in the hope to encourage the reuse of some of the code or strategies to tackle related problems.

## For the impatient

$\def\QQ{\mathbb{Q}}$
Let $P_{S_n}$ be the diagonal coinvariant ring for $k$ sets of $n$ variables. The $GL_k$ character contributed by a given irreducible representation of $S_n$ can be computed with
the following command; the result is fetched from database if already calculated:

In [2]:
%runfile code.py
%display unicode_art

Compiling ./code1.pyx...


In [5]:
harmonic_character([3,2])

s    + s    + s    + s     + s     + s      + s      + s       + s
 ┌┬┐    ┌┬┐    ┌┬┐    ┌┬┬┐    ┌┬┬┐    ┌┬┬┬┐    ┌┬┬┬┐    ┌┬┬┬┬┐    ┌┬┬┬┬┬┐
 └┴┘    ├┼┘    ├┼┤    └┴┴┘    ├┼┴┘    └┴┴┴┘    ├┼┴┴┘    └┴┴┴┴┘    └┴┴┴┴┴┘
        └┘     └┴┘            └┘               └┘             

This triggers the calculation of the above for all representations of $S_n$:

In [3]:
harmonic_characters(3)

[3]	(0:00:00): s[]
[2, 1]	(0:00:00): s[1] + s[2]
[1, 1, 1]	(0:00:00): s[1, 1] + s[3]


Compute all the currently computed terms of the $GL_r\times S_n$ character of diagonal harmonic polynomials:

In [42]:
harmonic_bicharacter_truncated_series()

1 # 1 + 1 # s   + 1 # s    + 1 # s     + 1 # s      + 1 # s       + 1 # 
             ┌┐        ┌┬┐        ┌┬┬┐        ┌┬┬┬┐        ┌┬┬┬┬┐       
             └┘        └┴┘        └┴┴┘        └┴┴┴┘        └┴┴┴┴┘       
                                                                        
                                                                        
                                                                        
                                                                        
                                                                        

s        + s   # s   + s   # s    + s   # s     + s   # s      + s   # s      
 ┌┬┬┬┬┬┐    ┌┐    ┌┐    ┌┐    ┌┬┐    ┌┐    ┌┬┬┐    ┌┐    ┌┬┬┬┐    ┌┐    ┌┬┬┬┬┐
 └┴┴┴┴┴┘    └┘    ├┤    └┘    ├┼┘    └┘    ├┼┴┘    └┘    ├┼┴┴┘    └┘    ├┼┴┴┴┘
                  └┘          └┘           └┘            └┘             └┘    
                                                                              
                    

In [6]:
%display latex

## Context

Note: throughout this notebook, we implicitly use the Frobenius map, and identify $S_n$ and $GL_k$ characters with their associated symmetric functions.

Consider the polynomial ring $P=\QQ[X]$ with $k$ sets of $n$ variables. To fix conventions, we think of $X$ as a matrix of variables:
$$(X_{i,j})_{0\leq i<k, 0\leq j<n}$$

The polynomial ring $P$ is endowed with an action of the symmetric group $S_n$ by diagonal permutation of the columns, and $GL_k$ by diagonal action on the columns. The invariant ring $P^{S_n}$ is also known as the ring of multisymmetric functions. The coinvariant ring $P_{S_n}$ is the quotient of $R$ by the ideal $\langle {P^{S_n}}^+ \rangle$. It inherits from $P$ a structure of graded $GL_k$-$S_n$ bimodule.

The grading can be refined into a multigrading by rows. The multigrading is preserved by the action of $S_n$. The multigrading is not preserved by the action of $GL_k$, but instead carries exactly the information about the $GL_r$ character. It's thus equivalent to consider the $GL_r$-$S_n$ bicharacter or the $S_n$ multigraded character of $P_{S_n}$.

When $k=1$, the action of $GL_1$ is trivial; $P_{S_n}$ is of dimension $n!$ and carries the usual graded representation of $S_n$. Its character is given by a Hall-Littlewood polynomial.
When $k=2$, $P_{S_n}$ is of dimension $(n+1)^{n-1}$ ($=16807$ for $n=6$); as a consequence of Haiman's proof of the $n!$-conjecture, the bigraded character is a Macdonald polynomial.

Knowing the bicharacter of $P_{S_n}$ for $n=6$ and $r=5$, we can recover its dimension:

In [16]:
n = 6
r = 5
d = sum(  sum(harmonic_character(mu).expand(r).coefficients())
    * StandardTableaux(mu).cardinality()
    for mu in Partitions(n)); d

720

Brute force computation would require intractable linear algebra on this space: $\gg d^3$ operations, meaning years of calculations:

In [18]:
d = 3.751076

In [19]:
d^3 / 10.^9 / 3600 / 24 / 365

1673.63590107261

(for simplicity we assumed in the above estimate a dense linear algebra implementation of complexity $O(n^3)$; in practice it's less than this, but on the other hand the calculation happens in the full ring of polynomials).

# Strategies

## Computing in the space of harmonic polynomials

We first need to compute in a quotient by an ideal.
A standard approach would be to compute a Gröbner basis of this ideal.
We haven't actually tried it, but given the number of variables and dimension of the quotient, we expect this calculation to be intractable.
Beside 

So we took the classical route of realizing the coinvariant ring as the space $\def\harmonics{\mathcal{H}}\harmonics_{k,n}$ of *diagonal harmonic polynomials*.
This is the space of those polynomials that satisfy the equations given by the multisymmetric functions seen as differential operators.

Solving those differential equations would not be that tractable either.

## Polarization: diagonal harmonic polynomials from harmonic polynomials

Define the *polarization operators*:
    
$$P_{i,i'}^d = \sum_j   x_{i',j} \partial_{i,j}^d\,.$$

For $d=1$, those operators implement the action of $sl_k$ induced by $gl_k$.

**Theorem:** Multisymmetric polynomials are obtained by polarization of symmetric polynomials in the first set of variables.

**Conjecture:** Diagonal harmonic polynomials are obtained by polarization of harmonic polynomials in the first set of variables.

In the sequel, we assume that the above conjecture holds. The space of harmonic polynomials $\harmonics_n$ itself can be constructed, for example, as the subspace spanned by the VanDerMonde determinant under differentiation.

In practice, we implemented a general purpose utility in Sage `Space(vectors, operators)` that computes the smallest subspace of a vector space which contains the given vectors and is stable under the action of the given linear operators.

**TODO**: put an example here

No rocket science here: maintain a matrix in row echelon form representing a basis of the space computed so far, and a ToDo list of $op(v)$ that remain to be computed and inserted in the matrix.

This utility should already have been in Sage, and will end up there.

Once a basis of $\harmonics_{k,n}$ is constructed, there still remains to recover its bicharacter. For $GL_k$ that's given by the multigrading. For $S_n$ some additional work has to be done (e.g. computing the traces of the action of permutations of all cycle types).

At this stage, we have an algorithm, but are still doing linear algebra in a huge space (cost $N^2 M$, where $N$ is the dimension of the result and $M$ the dimension of the ambient space).

# Exploiting the multigrading

The above utility can be adapted to exploit the multigrading. The input is now a collection of homogeneous vectors and operators together with their multidegrees, and a separate row echelon matrix is maintained for each multidegree.

Advantages:
- Writing $N=:\sum N_D$, The complexity becomes $(\sum N_D^2) M$ instead of $(\sum N_D)^2M_D$ (in fact better.
- The multidegrees, and therefore $GL_k$ character, can be directly read off the result.

## Exploiting representations of the symmetric group

The polarization operators commute with the action of $S_n$. Therefore,
we can compute independently each $S_n$ isotypic component of $\harmonics_{k,n}$ by starting from a basis of harmonic polynomials that itself is adapted to its decomposition into isotypic components:
$$\harmonics_{n}=:\bigoplus_\lambda \harmonics^\lambda\,.$$

In fact we can do better.

Fix a partition $\lambda$. Recall that a basis of the Specht module $V_\lambda$ is given by the family of Garnir/Specht polynomials $(G_{P})_P$ indexed by standard tableaux $P$ of shape $\lambda$.

To get a basis of the full isotypic component $\harmonics^\lambda$, we can, for example, use a generalization of this construction (called *Higher Specht Polynomials* in [Ariki, Terasoma, Yamada 97]): a family of polynomials $(G_{P,Q})_{P,Q}$ indexed by a pairs of standard tableaux of shape $\lambda$:
- $Q$ specifies the copy of $V_\lambda$
- $P$ specifies the location within that copy of $V_\lambda$


**Remark**: For a fixed tableau $P$, denote by $\harmonics_k^P$ the space spanned by the higher Specht polynomials $(G_{P,Q})_Q$ under polarization. It's exactly the image of $\harmonics$ under the Young idempotent associated to $P$. For a given shape $\lambda$, the $\harmonics_k^P$ are all conjugate under $S_n$, and their direct sum is the isotypic component $\harmonics_k^\lambda$.

**Conclusion:** It is sufficient to compute $\harmonics_k^P$ for one tableau of each shape $\lambda$. This reduces the complexity by the square of the number of standard tableaux of shape $\lambda$

In [20]:
%display unicode_art
{mu: StandardTableaux(mu).cardinality()^2 for mu in Partitions(6)}

⎧                                                                          
⎪                                              ┌┬┐                         
⎪         ┌┬┐                        ┌┬┬┐      ├┼┘                         
⎨ ┌┬┐     ├┼┤              ┌┬┬┐      ├┼┴┘      ├┤                          
⎪ ├┼┤     ├┼┘     ┌┬┬┐     ├┼┼┘      ├┤        ├┤                 ┌┬┬┬┐    
⎪ ├┼┤     ├┤      ├┼┼┤     ├┼┘       ├┤        ├┤      ┌┬┬┬┬┬┐    ├┼┼┴┘    
⎩ └┴┘:25, └┘ :81, └┴┴┘:25, └┘  :256, └┘  :100, └┘ :25, └┴┴┴┴┴┘:1, └┴┘  :81,

            ┌┐              ⎫
            ├┤              ⎪
            ├┤              ⎪
            ├┤    ┌┬┬┬┐     ⎬
 ┌┬┬┬┬┐     ├┤    ├┼┴┴┘     ⎪
 ├┼┴┴┴┘     ├┤    ├┤        ⎪
 └┘    :25, └┘:1, └┘   :100 ⎭

**Cherries on the cake:**
- The bicharacter can be read off directly from the graded dimensions of each $\harmonics_r^P$
- The calculations for each isotypic component can be carried independently and in parallel

## Exploiting column antisymmetries

Specht polynomials tend to be big: in $6$ variables, the VanDerMonde determinant has $6!=720$ monomials.

But that's mostly because they are antisymmetric w.r.t. a large column stabilizer $S_P$. It's therefore sufficient to keep one representative monomial per $S_P$ orbit (and do some appropriate straightening whenever a new monomial is created). 

This trick was already used by François for the polarizations of the VanDerMonde (the diagram space). Using higher Specht polynomials further allowed  to generalize that optimization to each shape.

**Gain**: in theory up to $n!$. In practice between one and two orders of magnitude due to the overhead of straightening monomials.


# Exploiting row symmetries

$H_k^P$ is symmetric w.r.t. permutation of rows, and such permutations
exchange multigraded components. For example, for $r=3$, the homogeneous components of respective multidegree $4,2,1$ and $2,1,4$ coincide
up to cycling of the rows. It is therefore sufficient, in principle, to compute homogeneous components with decreasing multidegree.
To achieve this, polarization operators can be tweaked: whenever the result of a polarization is not of decreasing multidegree, straighten it by applying an appropriate permutation of the rows. It can then easily be shown that those tweaked polarization operator properly generate the whole $\harmonics_k^P$ space from $\harmonics^P$.

**Gain**: an order of magnitude; below are the total dimension of each $\harmonics_5^\lambda$ and its restriction to homogeneous components of decreasing multidegree:

In [38]:
S = SymmetricFunctions(QQ); S.inject_shorthands(verbose=False)
def f(mu): return harmonic_character(mu)
{mu: [sum(m(f(mu)).expand(5).coefficients()), sum(m(f(mu)).coefficients())]
 for mu in Partitions(6)}

⎧                                                                             
⎪                                                                             
⎪                      ┌┬┐                                                    
⎨ ┌┬┐                  ├┼┤                                       ┌┬┬┐         
⎪ ├┼┤                  ├┼┘                   ┌┬┬┐                ├┼┼┘         
⎪ ├┼┤                  ├┤                    ├┼┼┤                ├┼┘          
⎩ └┴┘:[ 41825, 3193 ], └┘ :[ 119032, 9761 ], └┴┴┘:[ 5880, 351 ], └┘  :[ 46472,

                                                                       
                               ┌┬┐                                     
         ┌┬┬┐                  ├┼┘                                     
         ├┼┴┘                  ├┤                                      
         ├┤                    ├┤                                      
         ├┤                    ├┤                     ┌┬┬┬┬┬┐          
 3014 ], └┘  :

## Exploiting representations of $sl_k$

Building on the previous strategy, it would be tempting to only compute the subspace of $sl_r$ highest weight vectors within each $\harmonics_k^P$.

The current code partially achieves this by avoiding the use of polarization operators of degree $0$ (which clearly would leave the space of highest weight vectors). The obtained space is still too big, and the actual highest weight vectors are obtained by computing the anihilator of the polarization operators $P_{i,i-1}^1$.

The first step requires an additional, not yet proven, trick.

We are currently exploring how to tweak the polarization operators to stabilize the space of highest weight vectors.

**Current gain:** another order of magnitude; down from 3 days to 45 minutes for $\harmonics_5^{1^6}$

**Potential gain:** another order of magnitude, both in time and memory:

In [40]:
{mu: [sum(m(f(mu)).coefficients()), sum(f(mu).coefficients())]
 for mu in Partitions(6)}

⎧                                                                         
⎪                                                                         
⎪                   ┌┬┐                                                   
⎨ ┌┬┐               ├┼┤                                 ┌┬┬┐              
⎪ ├┼┤               ├┼┘               ┌┬┬┐              ├┼┼┘              
⎪ ├┼┤               ├┤                ├┼┼┤              ├┼┘               
⎩ └┴┘:[ 3193, 40 ], └┘ :[ 9761, 88 ], └┴┴┘:[ 351, 16 ], └┘  :[ 3014, 72 ],

                                                                           
                    ┌┬┐                                                    
 ┌┬┬┐               ├┼┘                                                    
 ├┼┴┘               ├┤                                                     
 ├┤                 ├┤                                   ┌┬┬┬┐             
 ├┤                 ├┤                 ┌┬┬┬┬┬┐           ├┼┼┴┘             
 └┘  :[ 4413, 63 ]

## Exploiting the skew commutation of polarization operators
The polarization operators skew commute. Therefore it would be in principle sufficient to apply them in some "decreasing order". This sounds promising given that, at this stage about 90% of the polynomials that are produced are redundant and reduced to zero by the linear algebra.

However we have not yet been able put to use that optimization together with the other $sl_k$ optimizations, because the tweaked polarization operators don't skew commute consistently.

# Some *not cleaned up* examples of use

## Computing the space of harmonics manually

In [16]:
n = 5
r = 2
P = DiagonalPolynomialRing(QQ,n,r)
X = P.algebra_generators(); X

[x00 x01 x02 x03 x04]
[x10 x11 x12 x13 x14]

In [17]:
Degrees = cartesian_product([ZZ]*r)
def add_degree((d1,mu), d2):
    d = d1+d2
    if not all(i>=0 for i in d):
        raise ValueError("Invalid degree")
    return d,mu

R = QQ['q,t']
Sym = SymmetricFunctions(R.fraction_field())
Sym.inject_shorthands()
def hilbert_parent(dimensions):
    return s.sum_of_terms( (mu, R({d:dim})) for (d,mu),dim in dimensions.iteritems() )

/opt/sage-git2/local/lib/python2.7/site-packages/sage/combinat/sf/sf.py:1416: RuntimeWarning: redefining global value `e`
  inject_variable(shorthand, getattr(self, shorthand)())


In [18]:
R({Degrees([2,1]):3})

3*q^2*t

In [19]:
dimensions = {(Degrees([2,1]), Partition([3,2])): 42,
              (Degrees([0,0]), Partition([2])): 5}
hilbert_parent(dimensions)

5*s[2] + 42*q^2*t*s[3, 2]

In [20]:
generators = {}
for mu in Partitions(n):
    for t in StandardTableaux(mu):
        p = P.higher_specht(t, harmonic=True)
        d = (Degrees([p.degree()]+[0]*(r-1)), mu)
        if d not in generators:
            generators[d] = []
        generators[d].append(p)
for key,value in generators.iteritems():
    print key, [factor(p) for p in value]

((5, 0), [3, 2]) [(2/25) * (x01 - x04) * (-x00 + x03) * (x00^3 - 2*x00^2*x01 - 2*x00*x01^2 + x01^3 - 2*x00^2*x02 + 6*x00*x01*x02 - 2*x01^2*x02 + 3*x00*x02^2 + 3*x01*x02^2 - 4*x02^3 + 3*x00^2*x03 + x00*x01*x03 - 2*x01^2*x03 - 14*x00*x02*x03 + 6*x01*x02*x03 + 3*x02^2*x03 + 3*x00*x03^2 - 2*x01*x03^2 - 2*x02*x03^2 + x03^3 - 2*x00^2*x04 + x00*x01*x04 + 3*x01^2*x04 + 6*x00*x02*x04 - 14*x01*x02*x04 + 3*x02^2*x04 + x00*x03*x04 + x01*x03*x04 + 6*x02*x03*x04 - 2*x03^2*x04 - 2*x00*x04^2 + 3*x01*x04^2 - 2*x02*x04^2 - 2*x03*x04^2 + x04^3)]
((3, 0), [3, 2]) [(4/5) * (x01 - x04) * (-x00 + x03) * (x00 + x01 - 4*x02 + x03 + x04)]
((4, 0), [3, 1, 1]) [(-2/5) * (x03 - x04) * (-x00 + x03) * (x00 - x04) * (2*x00 - 3*x01 - 3*x02 + 2*x03 + 2*x04)]
((6, 0), [3, 1, 1]) [(1/20) * (x03 - x04) * (-x00 + x03) * (x00 - x04) * (x00^2*x01 - 2*x01^3 + x00^2*x02 - 8*x00*x01*x02 + 6*x01^2*x02 + 6*x01*x02^2 - 2*x02^3 - x00^2*x03 + 3*x00*x01*x03 + 3*x00*x02*x03 - 8*x01*x02*x03 - x00*x03^2 + x01*x03^2 + x02*x03^2 - x00^2*x

In [21]:
operators = P.polarization_operators_by_degree(side="down"); operators

{(-3, 1): [<functools.partial object at 0x7f1bc22d2628>],
 (-1, 1): [<functools.partial object at 0x7f1bc22e5050>],
 (-4, 1): [<functools.partial object at 0x7f1bc22d27e0>],
 (-2, 1): [<functools.partial object at 0x7f1bc22d2788>]}

In [22]:
F = Subspace(generators, operators=operators, add_degrees=add_degree)
F._hilbert_parent = hilbert_parent

In [23]:
%time dims = F.dimensions()

CPU times: user 2.37 s, sys: 24 ms, total: 2.4 s
Wall time: 2.38 s


In [24]:
F.hilbert_polynomial()

(q^10+q^9*t+q^8*t^2+q^7*t^3+q^6*t^4+q^5*t^5+q^4*t^6+q^3*t^7+q^2*t^8+q*t^9+t^10+q^8*t+q^7*t^2+q^6*t^3+q^5*t^4+q^4*t^5+q^3*t^6+q^2*t^7+q*t^8+q^7*t+2*q^6*t^2+2*q^5*t^3+2*q^4*t^4+2*q^3*t^5+2*q^2*t^6+q*t^7+q^6*t+q^5*t^2+2*q^4*t^3+2*q^3*t^4+q^2*t^5+q*t^6+q^4*t^2+q^3*t^3+q^2*t^4)*s[1, 1, 1, 1, 1] + (q^9+q^8*t+q^7*t^2+q^6*t^3+q^5*t^4+q^4*t^5+q^3*t^6+q^2*t^7+q*t^8+t^9+q^8+2*q^7*t+2*q^6*t^2+2*q^5*t^3+2*q^4*t^4+2*q^3*t^5+2*q^2*t^6+2*q*t^7+t^8+q^7+3*q^6*t+4*q^5*t^2+4*q^4*t^3+4*q^3*t^4+4*q^2*t^5+3*q*t^6+t^7+q^6+3*q^5*t+4*q^4*t^2+5*q^3*t^3+4*q^2*t^4+3*q*t^5+t^6+2*q^4*t+3*q^3*t^2+3*q^2*t^3+2*q*t^4+q^3*t+q^2*t^2+q*t^3)*s[2, 1, 1, 1] + (q^8+q^7*t+q^6*t^2+q^5*t^3+q^4*t^4+q^3*t^5+q^2*t^6+q*t^7+t^8+q^7+2*q^6*t+2*q^5*t^2+2*q^4*t^3+2*q^3*t^4+2*q^2*t^5+2*q*t^6+t^7+q^6+3*q^5*t+4*q^4*t^2+4*q^3*t^3+4*q^2*t^4+3*q*t^5+t^6+q^5+3*q^4*t+4*q^3*t^2+4*q^2*t^3+3*q*t^4+t^5+q^4+2*q^3*t+3*q^2*t^2+2*q*t^3+t^4+q^2*t+q*t^2)*s[2, 2, 1] + (q^7+q^6*t+q^5*t^2+q^4*t^3+q^3*t^4+q^2*t^5+q*t^6+t^7+q^6+2*q^5*t+2*q^4*t^2+2*q^3*t^3+2*q^2

In [25]:
_ - s(e[n].nabla())

0

# Debugging stuff

In [ ]:
F.hilbert_polynomial()[Partition([2,2,1])]

In [ ]:
d = (Degrees([0,7]), Partition([2,2,1]))

In [ ]:
F._bases[d].cardinality()

In [ ]:
p1,p2 = F._bases[d]._basis

In [ ]:
p1/=1440;p1

In [ ]:
p2 /= 5040; p2

In [ ]:
R({d[0]:1})

In [ ]:
F.dimensions()[d]

In [ ]:
len(F._todo)

In [ ]:
F.dimensions??

In [ ]:
sum(basis.cardinality() for d, basis in F._bases.iteritems())

In [ ]:
Subspace._extensions

In [ ]:
sum(s(e[n].nabla()).coefficients())(q=1,t=1)

In [ ]:
{d: basis._matrix for d, basis in F._bases.iteritems()}